In [1]:
!pip install imblearn -i https://pypi.tuna.tsinghua.edu.cn/simple
!pip install optuna -i https://pypi.tuna.tsinghua.edu.cn/simple
!pip install catboost -i https://pypi.tuna.tsinghua.edu.cn/simple

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.0/226.0 kB 1.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 1.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.8/24.8 MB 6.5 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: joblib
    Found existing installation: joblib 0.14.1
    Uninstalling joblib-0.14.1:
      Successfully uninstalled joblib-0.14.1
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.24.2
    Uninstalling scikit-learn-0.24.2:
      Successfully uninstalled scikit-learn-0.24.2

[notice] A new release of pip available: 22.1.2 -> 23.0.1
[notice] To update, run: pip install --upgrade pip
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 2.6 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip available: 22.1.2 -> 23.0.1
[notice] To upd

In [2]:
import pandas as pd
import numpy as np
import optuna
import time
from sklearn import metrics
from tqdm import tqdm
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from imblearn.over_sampling import BorderlineSMOTE,SMOTE,SMOTENC 
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import MinMaxScaler,LabelEncoder

# 标准化处理
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.combine import SMOTEENN
from collections import Counter
from imblearn.ensemble import BalancedBaggingClassifier
from scipy.stats import entropy


In [3]:
train = pd.read_csv(r'data/data203856/train.csv')
test = pd.read_csv(r'data/data203856/test.csv')
test['label'] = np.nan
data = pd.concat([train,test])
data.head()

,id,income,age,experience_years,is_married,city,region,current_job_years,current_house_years,house_ownership,car_ownership,profession,label
0,train_0,0.098182,0.224138,0.70,0,256,2,0.500000,0.00,2,1,18,0.0
1,train_1,0.353248,0.120690,0.55,1,8,12,0.285714,0.75,2,0,48,1.0
2,train_2,0.021639,0.551724,0.55,0,75,2,0.571429,0.00,2,0,5,0.0
3,train_3,0.632453,0.310345,0.25,1,303,11,0.357143,0.25,2,0,3,1.0
4,train_4,0.643962,0.827586,0.90,1,13,7,0.428571,0.00,2,0,24,0.0


In [4]:
for i in data.columns:
    print(i,data[i].nunique())

id 240000
income 40200
age 59
experience_years 21
is_married 2
city 317
region 29
current_job_years 15
current_house_years 5
house_ownership 3
car_ownership 2
profession 51
label 2


In [5]:
origin_feature = list(data.columns[1:-1])

# 特征工程

In [6]:
# //求熵
def myEntro(x):
    """
        calculate shanno ent of x
    """
    x = np.array(x)
    x_value_list = set([x[i] for i in range(x.shape[0])])
    ent = 0.0
    for x_value in x_value_list:
        p = float(x[x == x_value].shape[0]) / x.shape[0]
        logp = np.log2(p)
        ent -= p * logp
    #     print(x_value,p,logp)
    # print(ent)
    return ent

    
# //求值的范围
def myRange(x):
    return pd.Series.max(x) - pd.Series.min(x)
    
feature = ['income', 'age', 'experience_years', 'is_married', 'city',
       'region', 'current_job_years', 'current_house_years', 'house_ownership',
       'car_ownership', 'profession']
    #    , 'current_job_years_precentage',
    #    'income_pre_age', 'income_pre_experience_years',
    #    'experience_years_pre_age']

cat_feature = ['is_married','city','region','house_ownership','car_ownership','profession']
#类别特征
num_feature = ['income', 'age', 'experience_years', 'current_job_years', 'current_house_years']#,'income_pre_age','income_pre_experience_years','experience_years_pre_age']
#连续特征
ways = ['mean','max','min','std','sum','median',myEntro,myRange]#,myRms,myMode,myQ25,myQ75,myQ10,myQ90]
#进行统计的方法

In [7]:
for cat in tqdm(cat_feature):
    data[cat+"_"+"count"] =  data.groupby(cat)['id'].transform('count')
    feature.append(cat+"_"+"count")

    data[cat+"_"+"label_mean"] = data.groupby(cat)['label'].transform('mean')
    feature.append(cat+"_"+"label_mean")

    for num in num_feature:
        for way in ways:
            data[num+'_'+cat+"_"+str(way)] = data.groupby(cat)[num].transform(way)
            feature.append(num+'_'+cat+"_"+str(way))


num_feature = list(set(feature) - set(cat_feature))
for i in tqdm(['income']):
    data[i+'_cut'] = pd.cut(data[i],bins=100,labels=range(100)).astype('int')
    feature.append(i+'_cut')
    cat_feature.append(i+'_cut')

for i in tqdm(['age']):
    data[i+'_cut'] = pd.cut(data[i],bins=10,labels=range(10)).astype('int')
    feature.append(i+'_cut')
    cat_feature.append(i+'_cut')

for i in tqdm(['experience_years']):
    data[i+'_cut'] = pd.cut(data[i],bins=5,labels=range(5)).astype('int')
    feature.append(i+'_cut')
    cat_feature.append(i+'_cut')




100%|██████████| 1/1 [00:00<00:00, 69.92it/s]


In [8]:
enc = LabelEncoder()
num = len(cat_feature)
for i in tqdm(range(num)):
    for j in range(i,num):
        name = str(cat_feature[i])+"_"+str(cat_feature[j])
        data[name] = data[cat_feature[i]].astype(str) + data[cat_feature[j]].astype(str)
        data[name] = enc.fit_transform(data[name]) 
        feature.append(name)
        cat_feature.append(name)

100%|██████████| 9/9 [00:20<00:00,  2.25s/it]


In [9]:
transfer = StandardScaler()
# 调用fit_transform （只需要处理连续特征）
num_feature = list(set(feature) - set(cat_feature))
data[num_feature] = transfer.fit_transform(data[num_feature])

transfer = MinMaxScaler()
data[num_feature] = transfer.fit_transform(data[num_feature])

特征工程参考:
https://my.oschina.net/u/4295464/blog/4842023

# 特征筛选

In [10]:
new_feature_list = []
for i in feature:
    if data[i].nunique()>1:
        new_feature_list.append(i)
    else:
        if i in cat_feature:
            cat_feature.remove(i)

In [11]:
train = data[data['label'].notna()]
test = data[data['label'].isna()]
# del data
train.shape,test.shape

((156000, 313), (84000, 313))

In [12]:
# 数据划分与预处理
# feature = set(new_feature_list)
X = train[new_feature_list]
Y = train.label

In [13]:
feature = new_feature_list
X = train[feature]
Y = train.label

# 模型建立

In [14]:
def train_model_classification(X, X_test, y, params, num_classes=2,
                               folds=None, model_type='lgb',
                               eval_metric='logloss', columns=None,
                               plot_feature_importance=False,
                               model=None, verbose=10000,
                               early_stopping_rounds=200,
                               splits=None, n_folds=3):
    """
    分类模型函数
    返回字典，包括： oof predictions, test predictions, scores and, if necessary, feature importances.
    :params: X - 训练数据， pd.DataFrame
    :params: X_test - 测试数据，pd.DataFrame
    :params: y - 目标
    :params: folds - folds to split data
    :params: model_type - 模型
    :params: eval_metric - 评价指标
    :params: columns - 特征列
    :params: plot_feature_importance - 是否展示特征重要性
    :params: model - sklearn model, works only for "sklearn" model type
    """
    start_time = time.time()
    global y_pred_valid, y_pred

    columns = X.columns if columns is None else columns
    X_test = X_test[columns]
    splits = folds.split(X, y) if splits is None else splits
    n_splits = folds.n_splits if splits is None else n_folds

    # to set up scoring parameters
    metrics_dict = {
        'logloss': {
            'lgb_metric_name': 'logloss',
            'xgb_metric_name': 'logloss',
            'catboost_metric_name': 'Logloss',
            'sklearn_scoring_function': metrics.log_loss
        },
        'lb_score_method': {
            'sklearn_scoring_f1': metrics.f1_score,  # 线上评价指标
            'sklearn_scoring_accuracy': metrics.accuracy_score,  # 线上评价指标
            'sklearn_scoring_auc': metrics.roc_auc_score
        },
    }
    result_dict = {}
    count = 0
    # out-of-fold predictions on train data
    oof = np.zeros(shape=(len(X), num_classes))
    # averaged predictions on train data
    prediction = np.zeros(shape=(len(X_test), num_classes))
    # list of scores on folds
    acc_scores=[]
    scores = []
    # feature importance
    feature_importance = pd.DataFrame()

    # split and train on folds
    for fold_n, (train_index, valid_index) in enumerate(splits):
        if verbose:
            print(f'Fold {fold_n + 1} started at {time.ctime()}')
        if type(X) == np.ndarray:
            X_train, X_valid = X[train_index], X[valid_index]
            y_train, y_valid = y[train_index], y[valid_index]
        else:
            X_train, X_valid = X[columns].iloc[train_index], X[columns].iloc[valid_index]
            y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
        # 在此进行数据采样处理
        # print("采样处理前,正负样本比例为",Counter(y_train),end = '\t')
        # smo_enn = SMOTENC(categorical_features=[i for i in range(len(feature)) if feature[i] in cat_feature],random_state=2023,n_jobs=-1)
        # X_train, y_train = smo_enn.fit_resample(X_train, y_train)
        # print("采样处理后,正负样本比例为",Counter(y_train))

        if model_type == 'lgb':
            model = LGBMClassifier(**params)
            model.fit(X_train, y_train,
                      eval_set=[(X_train, y_train), (X_valid, y_valid)],
                      eval_metric=metrics_dict[eval_metric]['lgb_metric_name'],
                      verbose=verbose,
                      early_stopping_rounds=early_stopping_rounds)

            y_pred_valid = model.predict_proba(X_valid)
            y_pred = model.predict_proba(X_test, num_iteration=model.best_iteration_)

        if model_type == 'xgb':
            model = xgb.XGBClassifier(**params)
            model.fit(X_train, y_train,
                      eval_set=[(X_train, y_train), (X_valid, y_valid)],
                      eval_metric=metrics_dict[eval_metric]['xgb_metric_name'],
                      verbose=bool(verbose),  # xgb verbose bool
                      early_stopping_rounds=early_stopping_rounds)
            y_pred_valid = model.predict_proba(X_valid)
            y_pred = model.predict_proba(X_test, ntree_limit=model.best_ntree_limit)
        if model_type == 'sklearn':
            model = model
            model.fit(X_train, y_train)
            y_pred_valid = model.predict_proba(X_valid)
            score = metrics_dict[eval_metric]['sklearn_scoring_function'](y_valid, y_pred_valid)
            print(f'Fold {fold_n}. {eval_metric}: {score:.4f}.')
            y_pred = model.predict_proba(X_test)

        if model_type == 'cat':
            model = CatBoostClassifier(iterations=20000, eval_metric=metrics_dict[eval_metric]['catboost_metric_name'],
                                       **params,
                                       loss_function=metrics_dict[eval_metric]['catboost_metric_name'])
            model.fit(X_train, y_train, eval_set=(X_valid, y_valid), 
                        cat_features=cat_feature, 
                        # num_features = num_feature,
                        #  pairwise_interactions=origin_feature,
                        use_best_model=True,
                      verbose=False)
            # model = BalancedBaggingClassifier(base_estimator=model,
            #                     sampling_strategy='auto',
            #                     replacement=False,
            #                     random_state=0)
            # model.fit(X_train, y_train)

            y_pre_train = model.predict_proba(X_train)
            y_pred_valid = model.predict_proba(X_valid)
            y_pred = model.predict_proba(X_test)


        oof[valid_index] = y_pred_valid
        # 评价指标
        acc_scores.append(
            metrics_dict['lb_score_method']['sklearn_scoring_accuracy'](y_valid, np.argmax(y_pred_valid, axis=1)))
        scores.append(
            metrics_dict['lb_score_method']['sklearn_scoring_auc'](y_valid, y_pred_valid[:,1]))
        print(acc_scores)
        print(scores)
        # if scores[-1]>0.94:
        #     prediction += y_pred
        #     count = count+1
        prediction += y_pred
        if model_type == 'lgb' and plot_feature_importance:
            # feature importance
            fold_importance = pd.DataFrame()
            fold_importance["feature"] = columns
            fold_importance["importance"] = model.feature_importances_
            fold_importance["fold"] = fold_n + 1
            feature_importance = pd.concat([feature_importance, fold_importance], axis=0)

        if model_type == 'xgb' and plot_feature_importance:
            # feature importance
            fold_importance = pd.DataFrame()
            fold_importance["feature"] = columns
            fold_importance["importance"] = model.feature_importances_
            fold_importance["fold"] = fold_n + 1
            feature_importance = pd.concat([feature_importance, fold_importance], axis=0)
    prediction /= n_splits
    print('CV mean score: {0:.4f}, std: {1:.4f}.'.format(np.mean(scores), np.std(scores)))

    result_dict['oof'] = oof
    result_dict['prediction'] = prediction
    result_dict['acc_scores'] = acc_scores
    result_dict['scores'] = scores


    if model_type == 'lgb' or model_type == 'xgb':
        if plot_feature_importance:
            feature_importance["importance"] /= n_splits
            cols = feature_importance[["feature", "importance"]].groupby("feature").mean().sort_values(
                by="importance", ascending=False)[:50].index

            best_features = feature_importance.loc[feature_importance.feature.isin(cols)]

            plt.figure(figsize=(16, 12))
            sns.barplot(x="importance", y="feature", data=best_features.sort_values(by="importance", ascending=False))
            plt.title('LGB Features (avg over folds)')
            plt.show()
            result_dict['feature_importance'] = feature_importance
    end_time = time.time()

    print("train_model_classification cost time:{}".format(end_time - start_time))
    return result_dict

In [ ]:
cat_params = {'learning_rate': 0.1, 'depth': 9, 'l2_leaf_reg': 10, 'bootstrap_type': 'Bernoulli',#'task_type': 'GPU',
              'od_type': 'Iter', 'od_wait': 50, 'random_seed': 11, 'allow_writing_files': False}

n_fold = 10
num_classes = 2
print("分类个数num_classes:{}".format(num_classes))
folds = StratifiedKFold(n_splits=n_fold, random_state=1314, shuffle=True)


result_dict_cat = train_model_classification(X=X,
                                             X_test=test[feature],
                                             y=Y,
                                             params=cat_params,
                                             num_classes=num_classes,
                                             folds=folds,
                                             model_type='cat',
                                             eval_metric='logloss',
                                             plot_feature_importance=True,
                                             verbose=1,
                                             early_stopping_rounds=800,#原400
                                             n_folds=n_fold)

分类个数num_classes:2
Fold 1 started at Thu Apr 13 13:08:56 2023
[0.9032051282051282]
[0.940374324805705]
Fold 2 started at Thu Apr 13 13:11:27 2023
[0.9032051282051282, 0.9058333333333334]
[0.940374324805705, 0.9433745066510744]
Fold 3 started at Thu Apr 13 13:14:55 2023
[0.9032051282051282, 0.9058333333333334, 0.9024358974358975]
[0.940374324805705, 0.9433745066510744, 0.9367520694627971]
Fold 4 started at Thu Apr 13 13:17:20 2023
[0.9032051282051282, 0.9058333333333334, 0.9024358974358975, 0.9044871794871795]
[0.940374324805705, 0.9433745066510744, 0.9367520694627971, 0.9416194244693141]
Fold 5 started at Thu Apr 13 13:20:37 2023
[0.9032051282051282, 0.9058333333333334, 0.9024358974358975, 0.9044871794871795, 0.9055769230769231]
[0.940374324805705, 0.9433745066510744, 0.9367520694627971, 0.9416194244693141, 0.9405878127004588]
Fold 6 started at Thu Apr 13 13:24:13 2023
[0.9032051282051282, 0.9058333333333334, 0.9024358974358975, 0.9044871794871795, 0.9055769230769231, 0.9019230769230769

In [25]:
sub = pd.read_csv('cat_sub_5cv_cut.csv')
sub['label'] = result_dict_cat['prediction'][:, 1]
sub[['id','label']].to_csv('cat_sub_10cv.csv', index=None)

### 0.9395127068770044

Fold 5 started at Thu Apr  6 10:02:58 2023

[0.8665384615384616, 0.8759294871794872, 0.8741346153846153, 0.8936858974358974, 0.9008012820512821]

[0.9079589677791218, 0.9277018656813405, 0.9215184274669903, 0.9364011520096265, 0.9361199423063424]

CV mean score: 0.9259, std: 0.0106.

### 0.9405441328372578

[0.903301282051282, 0.9008333333333334, 0.9008974358974359, 0.9055448717948718, 0.9026923076923077]

[0.9400705898728445, 0.9370555695899808, 0.9374504526928386, 0.9419737474623144, 0.9400199148017934]

CV mean score: 0.9393, std: 0.0018.

### 0.9406685678115321

[0.9053846153846153, 0.901474358974359, 0.9013461538461538, 0.9040384615384616, 0.9021794871794871]

[0.9400077373638717, 0.9374313607312934, 0.9378180336715911, 0.9412380521299796, 0.9394537118933349]

CV mean score: 0.9392, std: 0.0014.

[0.9362484888771758, 0.9382271433098509, 0.940827013838387, 0.9414658352487052, 0.9340132236489217, 0.9360767036876199, 0.9419637379298802, 0.939804305116714, 0.9423374126345797, 0.934654941171575, 0.9370173304886397, 0.9335991557984971, 0.9371071861845076, 0.9439558059354756, 0.9435747136939572, 0.9435345851608188, 0.938556971856725, 0.942038331505848, 0.9387929459064328, 0.938846780579922]
[0.9370931057436157, 0.9408271472119812, 0.9361387795691163, 0.940195185015088, 0.939051580227943, 0.935351245202654, 0.9391370534419747, 0.9429832528492308, 0.9397600289999494, 0.94030431130472]
CV mean score: 0.9391, std: 0.0022.


500early

[0.9020192307692307, 0.9015064102564102, 0.9009935897435898, 0.9035897435897436, 0.9016346153846154]

[0.9384516829695065, 0.936771090314636, 0.9364218869561385, 0.9407541573401077, 0.9392777185843183]

CV mean score: 0.9383, std: 0.0016.

MaxMin
[0.9395301859246554, 0.9373190948541511, 0.9368598924615383, 0.9411030130525107, 0.9390176328117195]
CV mean score: 0.9388, std: 0.0015.

只用income分箱

[0.9044551282051282, 0.9023717948717949, 0.90125, 0.9046794871794872, 0.9012820512820513]

[0.939636175122996, 0.9377035723855507, 0.9373895336676241, 0.9415378468825828, 0.9393120754644683]

CV mean score: 0.9391, std: 0.0015.


请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 